In [1]:
# Cell 1

# If not installed, you may need:
# !pip install pathway transformers torch scikit-learn

import sqlite3
import json
import torch
import pathway as pw

# Import VectorStoreServer + VectorStoreClient from Pathway
from pathway.xpacks.llm.vector_store import VectorStoreServer, VectorStoreClient

from transformers import AutoTokenizer, AutoModel

DB_PATH = "../db/research_papers.db"

# Conferences of interest
TARGET_CONFERENCES = ["CVPR", "NeurIPS", "EMNLP", "TMLR", "KDD"]


In [2]:
# Cell 2

import sqlite3
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel

model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def scibert_embedder(doc_text: str) -> list[float]:
    """
    Pathway-compatible embedder: takes a string and returns a list of floats.
    We'll do mean pooling over the last_hidden_state.
    """
    inputs = tokenizer(doc_text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Mean pooling across the sequence dimension
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().tolist()
    return embedding

def fetch_reference_papers_df():
    """
    Fetch from the same labelled_data table:
      1) publishable = 1
      2) conference in ("NeurIPS","KDD","TMLR","EMNLP","CVPR")
    Return as a Pandas DataFrame with columns: [id, conference, sections].
    """
    conf_list = ["NeurIPS", "KDD", "TMLR", "EMNLP", "CVPR"]
    placeholders = ", ".join(["?"] * len(conf_list))

    query = f"""
        SELECT id, conference, sections
        FROM labelled_data
        WHERE publishable = 1
          AND conference IN ({placeholders})
    """

    conn = sqlite3.connect("../db/research_papers.db")  # Adjust if needed
    df_ref = pd.read_sql_query(query, conn, params=conf_list)
    conn.close()
    return df_ref

df_references = fetch_reference_papers_df()
print(f"Fetched {len(df_references)} references as a DataFrame.")
df_references.head()


Fetched 263 references as a DataFrame.


,id,conference,sections
0,3808_The_Distortion_of_Binomia,NeurIPS,"{""output"": ""The Distortion of Binomial Voting ..."
1,461_LithoBench_Benchmarking_AI,NeurIPS,"{""output"": ""LithoBench: Benchmarking AI Comput..."
2,9310_Multi_task_learning_with_,NeurIPS,"{""output"": ""Multi-Task Learning with Summary S..."
3,557_EmbodiedGPT_Vision_Languag,NeurIPS,"{""output"": ""EmbodiedGPT: Vision-Language Pre-T..."
4,10107_Finite_Population_Regres,NeurIPS,"{""output"": ""Finite Population Regression Adjus..."


In [3]:
# Cell 3 (Updated)

import json
import pandas as pd
import pathway as pw
from pathway.xpacks.llm.vector_store import VectorStoreServer

def create_reference_pathway_table_from_df(df_ref: pd.DataFrame) -> pw.Table:
    """
    Transform our reference DataFrame into a Pathway table 
    with columns named [data, _metadata] (required by VectorStoreServer).
    Each row's `data` is the "output" field from your JSON,
    `_metadata` stores {paper_id, conference_label}.
    """
    rows_list = []

    for _, row in df_ref.iterrows():
        paper_id = row["id"]
        conf_label = row["conference"]
        sections_json = row["sections"]

        try:
            # Parse JSON
            sections_dict = json.loads(sections_json) if sections_json else {}
            # Extract the "output" field
            text_str = sections_dict.get("output", "")
            
            # Build metadata
            metadata_dict = {
                "paper_id": paper_id,
                "conference_label": conf_label
            }
            
            # The server expects columns named 'data' and '_metadata'
            rows_list.append((text_str, metadata_dict))
        except Exception as e:
            print(f"Skipping paper {paper_id} due to error: {e}")

    # Create a Pandas DataFrame with columns ["data", "_metadata"]
    df_pathway = pd.DataFrame(rows_list, columns=["data", "_metadata"])
    
    # Convert to a Pathway table
    table = pw.debug.table_from_pandas(df_pathway)
    return table

# Build the Pathway table
reference_table = create_reference_pathway_table_from_df(df_references)

def no_op_parser(contents):
    """
    A custom parser that returns a list of (text, metadata) 
    but does no actual parsing, because we already have text.
    """
    if isinstance(contents, str):
        # Already a string, just wrap in a list[tuple[str, dict]]
        return [(contents, {})]
    else:
        # If it arrives in bytes, decode as UTF-8
        return [(contents.decode("utf-8", errors="replace"), {})]


# Now create the VectorStoreServer using the correct columns
server = VectorStoreServer(
    reference_table,        # pass as a positional argument
    embedder=scibert_embedder,
    parser=no_op_parser,            # We already have 'data' in the doc rows
    splitter=None,
    doc_post_processors=None
)

server.run_server(host="0.0.0.0", port=8000, threaded=True, with_cache=True)
print("VectorStoreServer is running at http://0.0.0.0:8000")


/Users/advay/Projects/iitkgp-rpeval/.venv/lib/python3.11/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Sequence[str] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


VectorStoreServer is running at http://0.0.0.0:8000


======== Running on http://0.0.0.0:8000 ========
(Press CTRL+C to quit)


In [7]:
# Cell 4 (REPLACE YOUR EXISTING CELL 4 WITH THIS)

import json
import sqlite3
from typing import Tuple, List, Dict

# You must have installed openai and set your credentials/keys appropriately
from openai import OpenAI  

from pathway.xpacks.llm.vector_store import VectorStoreClient

# Initialize the Pathway VectorStoreClient
client = VectorStoreClient(host="0.0.0.0", port=8000, timeout=120)

# Initialize the GPT-4o-mini client via OpenAI (adjust if needed)
openai_client = OpenAI()

def generate_gpt_justification(
    paper_text: str,
    best_conference: str,
    reference_list: List[str]
) -> str:
    """
    Uses 'gpt-4o-mini' model via OpenAI to generate a concise justification
    (<=100 words) explaining how the paper's content, methodology, and findings
    align with the chosen conference's scope and quality standards.
    References are mentioned to illustrate alignment with related work.
    """
    system_command = (
        "You are an academic writing assistant. Given a paper's text, "
        "the recommended conference, and references from similar research, "
        "create a concise justification under 100 words explaining why "
        "this paper fits that venue’s scope and standards."
    )

    # Truncate paper text if large
    truncated_text = paper_text[:500]

    user_prompt = (
        f"Paper Text Excerpt: {truncated_text}\n"
        f"Recommended Conference: {best_conference}\n"
        f"Reference IDs: {reference_list}\n"
        "Please provide a concise alignment justification in under 100 words."
    )

    completion = openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_command},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.7,
    )

    justification = completion.choices[0].message.content

    # Enforce ~100-word limit
    words = justification.split()
    if len(words) > 100:
        justification = " ".join(words[:100])

    return justification

def fetch_publishable_papers() -> List[tuple]:
    """
    Load only 'publishable' papers (publishable=1) from labelled_data.
    Returns a list of (paper_id, file_name, sections_json).
    """
    conn = sqlite3.connect("../db/research_papers.db")
    cur = conn.cursor()
    cur.execute("SELECT id, file_name, sections FROM labelled_data WHERE publishable = 1")
    rows = cur.fetchall()
    conn.close()
    return rows

def classify_publishable_paper(paper_text: str, k=3) -> Tuple[str, str]:
    """
    Queries the vector store with the paper text, gets top-k matches,
    chooses the best conference by majority, then uses GPT-4o-mini 
    for a justification. Returns (conference_label, justification).
    """
    results = client.query(query=paper_text, k=k)
    conference_counts: Dict[str, int] = {}

    for r in results:
        conf_label = r["metadata"]["conference_label"]
        conference_counts[conf_label] = conference_counts.get(conf_label, 0) + 1

    if not conference_counts:
        return "Unknown", "No references found to determine conference."

    best_conference = max(conference_counts, key=conference_counts.get)
    reference_list = [
        f"{r['metadata']['paper_id']}({r['metadata']['conference_label']})" for r in results
    ]

    # Generate GPT-based rationale
    justification = generate_gpt_justification(paper_text, best_conference, reference_list)
    return best_conference, justification

def classify_all_publishable_papers() -> List[tuple]:
    """
    Classify all publishable papers, returning (paper_id, file_name, conf_label, justification).
    """
    results = []
    publishable_papers = fetch_publishable_papers()
    for (paper_id, file_name, sections_json) in publishable_papers:
        try:
            sections = json.loads(sections_json)
            paper_text = " ".join(sections.values()) if isinstance(sections, dict) else ""
            if not paper_text.strip():
                continue
            conf_label, justification = classify_publishable_paper(paper_text, k=3)
            results.append((paper_id, file_name, conf_label, justification))
        except Exception as e:
            print(f"Skipping paper {paper_id} due to error: {e}")
    return results

# Run the classification once
# classified_publishable = classify_all_publishable_papers()
# print(f"Classified {len(classified_publishable)} 'publishable' papers.")


In [8]:
# Cell 5 (MODIFY YOUR EXISTING CELL 5 TO THIS)

# Display the first 5 classifications
# for (paper_id, file_name, conf_label, justification) in classified_publishable[:5]:
#     print("="*50)
#     print(f"Paper ID: {paper_id}, File: {file_name}")
#     print(f"Recommended Conference: {conf_label}")
#     print(f"Justification: {justification}\n")

# (Optional) Write to a DB table
def store_classification_results(results):
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS recommended_conferences (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            paper_id INTEGER,
            file_name TEXT,
            recommended_conf TEXT,
            justification TEXT
        )
    """)
    for paper_id, file_name, conf_label, justification in results:
        cur.execute("""
            INSERT INTO recommended_conferences (paper_id, file_name, recommended_conf, justification)
            VALUES (?, ?, ?, ?)
        """, (paper_id, file_name, conf_label, justification))
    conn.commit()
    conn.close()

# Uncomment if you wish to store in DB
# store_classification_results(classified_publishable)


In [ ]:
# Cell 6: Evaluate classification accuracy on publishable papers
import sqlite3
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

# Suppose you used the same DB_PATH as before
DB_PATH = "../db/research_papers.db"

# 1) Convert your classification results list into a DataFrame
# classified_publishable is of the form: [(paper_id, file_name, conf_label, justification), ...]
df_pred = pd.DataFrame(
    classified_publishable, 
    columns=["paper_id", "file_name", "predicted_conf", "justification"]
)

# 2) Fetch true conference labels for these papers
conn = sqlite3.connect(DB_PATH)
df_gold = pd.read_sql_query(
    """
    SELECT id AS paper_id, conference AS true_conf
    FROM labelled_data
    WHERE publishable = 1
      AND conference IN ("NeurIPS","KDD","TMLR","EMNLP","CVPR")
    """,
    conn
)
conn.close()

# 3) Merge the predicted DataFrame (df_pred) with df_gold on paper_id
df_merged = pd.merge(df_pred, df_gold, on="paper_id", how="inner")

# 4) Compute accuracy
df_merged["correct"] = df_merged["predicted_conf"] == df_merged["true_conf"]
accuracy = df_merged["correct"].mean()  # True = 1.0, so mean is accuracy

print(f"Total Papers Evaluated: {len(df_merged)}")
print(f"Accuracy: {accuracy*100:.2f}%")

# 5) Print confusion matrix & classification report
TARGET_CONFERENCES = ["NeurIPS","KDD","TMLR","EMNLP","CVPR"]

y_true = df_merged["true_conf"]
y_pred = df_merged["predicted_conf"]

print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred, labels=TARGET_CONFERENCES))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, labels=TARGET_CONFERENCES))


In [9]:
# Cell 7: Classify unlabelled_data and update results table
def update_unlabelled_classification():
    """
    1) Fetch rows from unlabelled_data.
    2) Strip the '.pdf' from file_name => 'paper_id'.
    3) Check results table's publishable value for that paper_id.
    4) If publishable=1, classify and update (conference, rationale).
       Else set (conference, rationale) = ('na', 'na').
    """
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    # Ensure results table exists
    cur.execute("""
        CREATE TABLE IF NOT EXISTS results (
            paper_id TEXT PRIMARY KEY,
            publishable INTEGER,
            conference TEXT,
            rationale TEXT
        )
    """)

    # Fetch all unlabelled papers
    unlabelled_rows = cur.execute("SELECT id, file_name, sections FROM unlabelled_data").fetchall()

    for (u_id, file_name, sections_json) in unlabelled_rows:
        # Remove '.pdf' from file_name to get paper_id
        paper_id = file_name.replace(".pdf", "")

        # Check publishable status in results
        cur.execute("SELECT publishable FROM results WHERE paper_id = ?", (paper_id,))
        row = cur.fetchone()
        if not row:
            # No matching row in results, skip or create one if needed
            continue

        publishable_flag = row[0]
        if publishable_flag == 1:
            # Classify
            try:
                sections_dict = json.loads(sections_json) if sections_json else {}
                paper_text = sections_dict.get("output", "")
                conf_label, justification = classify_publishable_paper(paper_text, k=3)
            except Exception as e:
                print(f"Skipping classification for {paper_id} due to error: {e}")
                conf_label, justification = "na", "na"
        else:
            # Not publishable => set "na"
            conf_label, justification = "na", "na"

        # Update the results table
        cur.execute(
            "UPDATE results SET conference = ?, rationale = ? WHERE paper_id = ?",
            (conf_label, justification, paper_id)
        )

    conn.commit()
    conn.close()
    print("Finished updating unlabelled_data classifications in results table.")

# Finally, call this function:
update_unlabelled_classification()


Finished updating unlabelled_data classifications in results table.


In [10]:
# Download results.csv

def export_results_to_csv():
    """
    1) Connect to DB and fetch all rows from the 'results' table.
    2) Sort by paper_id in ascending order (R001, R002, R003, etc.).
    3) Convert conference names to lowercase.
    4) Save to results.csv in local directory (no index column).
    """
    import sqlite3
    import pandas as pd

    conn = sqlite3.connect(DB_PATH)
    # If paper_id always has format 'Rxxx' with leading zeros, a simple ORDER BY paper_id is fine:
    # e.g. 'R001', 'R010', 'R100'. Otherwise, consider numeric ordering, e.g.
    # "ORDER BY CAST(substr(paper_id, 2) AS INTEGER)"
    query = """
        SELECT paper_id, publishable, conference, rationale
        FROM results
        ORDER BY paper_id
    """
    df = pd.read_sql_query(query, conn)
    conn.close()

    # Convert conference to lowercase
    df["conference"] = df["conference"].str.lower()

    # Save to CSV
    df.to_csv("results.csv", index=False)
    print("results.csv saved locally, conferences converted to lowercase.")

# Call the function to perform export
export_results_to_csv()


results.csv saved locally, conferences converted to lowercase.
